In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:45707")
client

<Client: 'tcp://127.0.0.1:45707' processes=10 threads=50, memory=196.73 GiB>

In [2]:
client.restart()

<Client: 'tcp://127.0.0.1:45707' processes=0 threads=0, memory=0 B>

In [3]:
import numpy as np
import xarray as xr
from MITgcmutils import rdmds
from xmitgcm import open_mdsdataset
import os.path as op
import os
import gc
from scipy.interpolate import griddata, interp1d
# from multiprocessing import Pool
import matplotlib.pyplot as plt
%matplotlib inline

In [48]:
ddir = '/tank/chaocean/qjamet/'
dir_grd50 = '/tank/spectre/gridMIT_50/'
dir_obcs50 = '/tank/spectre/tuchida/obcs_50'
gdir = '/tank/chaocean/grid_chaO/gridMIT_update1/'

In [6]:
ds12 = open_mdsdataset(op.join(ddir,'RUNS/ORAR/memb%02d/run%4d/ocn/' 
                               % (0,2003)), 
                       grid_dir=gdir,
                       iters=7097760,
                       prefix=['diag_ocnTave'], delta_t=2e2,
                      )
ds12

<xarray.Dataset>
Dimensions:  (time: 1, XC: 1000, YC: 900, XG: 1000, YG: 900, Z: 46, Zp1: 47,
              Zu: 46, Zl: 46)
Coordinates: (12/33)
    iter     (time) int64 7097760
  * time     (time) timedelta64[ns] 16430 days
  * XC       (XC) >f4 262.0 262.1 262.2 262.3 262.3 ... 347.7 347.8 347.9 348.0
  * YC       (YC) >f4 -19.92 -19.83 -19.75 -19.67 ... 54.75 54.83 54.92 55.0
  * XG       (XG) >f4 262.0 262.0 262.1 262.2 262.3 ... 347.7 347.8 347.8 347.9
  * YG       (YG) >f4 -19.96 -19.87 -19.79 -19.71 ... 54.71 54.79 54.88 54.96
    ...       ...
    maskW    (Z, YC, XG) bool dask.array<chunksize=(46, 900, 1000), meta=np.ndarray>
    maskS    (Z, YG, XC) bool dask.array<chunksize=(46, 900, 1000), meta=np.ndarray>
    maskInC  (YC, XC) bool dask.array<chunksize=(900, 1000), meta=np.ndarray>
    rhoRef   (Z) >f4 1.021e+03 1.021e+03 1.021e+03 ... 1.044e+03 1.044e+03
    maskInW  (YC, XG) bool dask.array<chunksize=(900, 1000), meta=np.ndarray>
    maskInS  (YG, XC) bool dask.array<chunksize=(900, 1000), meta=np.ndarray>
Data variables:
    THETA    (time, Z, YC, XC) >f4 dask.array<chunksize=(1, 46, 900, 1000), meta=np.ndarray>
    SALT     (time, Z, YC, XC) >f4 dask.array<chunksize=(1, 46, 900, 1000), meta=np.ndarray>
    UVEL     (time, Z, YC, XG) >f4 dask.array<chunksize=(1, 46, 900, 1000), meta=np.ndarray>
    VVEL     (time, Z, YG, XC) >f4 dask.array<chunksize=(1, 46, 900, 1000), meta=np.ndarray>
    WVEL     (time, Zl, YC, XC) >f4 dask.array<chunksize=(1, 46, 900, 1000), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    title:        netCDF wrapper of MITgcm MDS binary data
    source:       MITgcm
    history:      Created by calling `open_mdsdataset(grid_dir='/tank/chaocea...

In [16]:
#-- some parameters --
nt = 73         #also need earlier and past year for interpolation, 
                #for now it is coded in a 'normal year' fashion
# nproc = 36      #number of processors used for parallelization 

#-- time param for loading --
dt = 200                #1/12 model time step
spy = 86400*365
dump = 5*86400       #5-d dumps
d_iter = dump/dt
nDump = spy/dump;

rSphere = 6370000.0

In [19]:
rC12 = ds12.Z
rF12 = ds12.Zl
[nr12] = rC12[:].shape
xx12 = rdmds(gdir + 'XC')
[ny12, nx12]  = xx12.shape
zzz12 = np.zeros([nr12+2])
zzz12[1:-1] = rC12[:]
zzz12[-1] = rF12[-1]
#- 1/50 -
rC50 = rdmds(dir_grd50 + 'RC')[:, 0, 0]
[nr50] = rC50.shape
xx50 = rdmds(dir_grd50 + 'XC')
[ny50, nx50]  = xx50.shape

#-- variables and boundaries --
varN = ['t', 's', 'uE', 'vN']
nvar = len(varN)
varOrder = [0, 1, 2, 3]         #Ordering of variables in diagnostic used to make obcs
varName = ['THETA','SALT','UVEL','VVEL']
bbdy = ['south', 'north', 'east', 'west']
nnbdy = len(bbdy)

In [37]:
#-----------------------------------------------------
# Define horizontal and vertical interpolation methods 
#-----------------------------------------------------

def interp_obcs(tttt):
  #print("-- tt=%02i || Interpolate %s at bdy %s --" % (tttt, varN[ivar], bbdy[ibdy]))
    tmpvarin = var12[tttt, :, :, :].reshape([nr12, ny2*nx2])
  #- hz interp -
    tmpvar = np.zeros([nr12+2, nxy50])
    for kkk in range(nr12):
        tmpvar[kkk+1] = griddata(xy12, tmpvarin[kkk, :], (xx50, yy50), method=mmeth)
    tmpvar[0, :] = tmpvar[1, :]
    tmpvar[-1, :] = tmpvar[-2, :]
  #- vert interp -
    tmpvar2 = np.zeros([nr50, nxy50])
    for ij in range(nxy50):
    # FOR TRACER ONLY
    # find last wet point and repeat it downward for constant interpolation 
        if varN[ivar] == 't' or varN[ivar] == 's':
            tmpk = np.where( tmpvar[:, ij] == 0.0 )[0]
            if tmpk.size > 0:
                if (tmpk[0] > 0 and tmpk[0] < (nr12+1) ):
                    tmpvar[tmpk[0]:, ij] = tmpvar[tmpk[0]-1, ij]
    #
        f = interp1d(zzz12, tmpvar[:, ij], fill_value='extrapolate')
        tmpvar2[:, ij] = f(rC50)
    return tmpvar2

In [49]:
iper = 2003
offset = int((iper-1958)*spy/dt)
# iters = np.arange(d_iter, (nDump+1)*d_iter, d_iter, dtype='int') + offset
iters3 = (np.arange(d_iter, (nDump+1)*d_iter, d_iter, dtype='int') + offset)
iters3 = iters3[:len(iters3)//2]
offset = int(((iper-1)-1958)*spy/dt)
iters2 = (np.arange(d_iter, (nDump+1)*d_iter, d_iter, dtype='int') + offset)
iters2 = iters2[len(iters2)//2:]
print(iters3, iters2)
tmpdir = str('%s/repeat%04i' % (dir_obcs50, iper))
if not op.isdir(tmpdir):
    os.makedirs( tmpdir )

[7097760 7099920 7102080 7104240 7106400 7108560 7110720 7112880 7115040
 7117200 7119360 7121520 7123680 7125840 7128000 7130160 7132320 7134480
 7136640 7138800 7140960 7143120 7145280 7147440 7149600 7151760 7153920
 7156080 7158240 7160400 7162560 7164720 7166880 7169040 7171200 7173360] [7017840 7020000 7022160 7024320 7026480 7028640 7030800 7032960 7035120
 7037280 7039440 7041600 7043760 7045920 7048080 7050240 7052400 7054560
 7056720 7058880 7061040 7063200 7065360 7067520 7069680 7071840 7074000
 7076160 7078320 7080480 7082640 7084800 7086960 7089120 7091280 7093440
 7095600]


In [53]:
#------------------------------------------------------------------
#       Make Boundary conditions from our previous 1/12 runs
#------------------------------------------------------------------

#-- Loop over variables and boundaries --
for ivar in range(nvar):
    if varN[ivar] == 'uE':
        flag_repz = False   #to repeat grid points downward for vert. interp
        mmeth = 'cubic'     #interpolation method
    # 1/50
        x50deg = rdmds(dir_grd50 + 'XG')
        y50deg = rdmds(dir_grd50 + 'YC')
        hFac50 = rdmds(dir_grd50 + 'hFacW')
    # 1/12
        x12deg = rdmds(gdir + 'XG')
        y12deg = rdmds(gdir + 'YC')
    elif varN[ivar] == 'vN':
        flag_repz = False   #to repeat grid points downward for vert. interp
        mmeth = 'cubic'     #interpolation method
    # 1/50
        x50deg = rdmds(dir_grd50 + 'XC')
        y50deg = rdmds(dir_grd50 + 'YG')
        hFac50 = rdmds(dir_grd50 + 'hFacS')
    # 1/12
        x12deg = rdmds(gdir + 'XC')
        y12deg = rdmds(gdir + 'YG')
    else:         # tracers
        flag_repz = True    #to repeat grid points downward for vert. interp
        mmeth = 'linear'    #interpolation method
    # 1/50
        x50deg = rdmds(dir_grd50 + 'XC')
        y50deg = rdmds(dir_grd50 + 'YC')
        hFac50 = rdmds(dir_grd50 + 'hFacC')
    # 1/12
        x12deg = rdmds(gdir + 'XC')
        y12deg = rdmds(gdir + 'YC')
  # define associated grid in [m] with co-localized origin
    tmpx50 = np.radians(x50deg - x50deg[0,0]) * rSphere * np.cos(np.radians(y50deg))
    tmpy50 = np.radians(y50deg - y50deg[0,0]) * rSphere
    tmpx12 = np.radians(x12deg - x50deg[0,0]) * rSphere * np.cos(np.radians(y12deg))
    tmpy12 = np.radians(y12deg - y50deg[0,0]) * rSphere
  # need to be consistent with ordering in bbdy
    [sbdy, nbdy, wbdy, ebdy] = [y50deg.min(), y50deg.max(), x50deg.min(), x50deg.max()]
  #
    for ibdy in range(nnbdy):
        print("== Deal with %s at bdy %s ==" % (varN[ivar], bbdy[ibdy]) )
        #
        #-- boundary parameters --
        deltaxy = 3 #+/- deltaxy grid points (on the 1/12 grid) around boundary
        if bbdy[ibdy] == 'south':
      #- 1/50 -
            if varN[ivar] == 'vN': #obcs are applied at inner grid points
                msk50 = hFac50[:, 1, :]
                xx50 = tmpx50[1, :]
                yy50 = tmpy50[1, :]
            else:
                msk50 = hFac50[:, 0, :]
                xx50 = tmpx50[0, :]
                yy50 = tmpy50[0, :]
            nxy50 = nx50
      #-  associated subdomain on the 1/12 grid -
            iiw = np.where(x12deg[0,:]>wbdy)[0][0] - deltaxy
            iie = np.where(x12deg[0,:]>ebdy)[0][0] + deltaxy
            jjs = np.where(y12deg[:,0]>sbdy)[0][0] - deltaxy
            jjn = np.where(y12deg[:,0]>sbdy)[0][0] + deltaxy
        elif bbdy[ibdy] == 'north':
      #- 1/50 -
            msk50 = hFac50[:, -1, :]
            xx50 = tmpx50[-1, :]
            yy50 = tmpy50[-1, :]
            nxy50 = nx50
      #-  associated subdomain on the 1/12 grid -
            iiw = np.where(x12deg[0,:]>wbdy)[0][0] - deltaxy
            iie = np.where(x12deg[0,:]>ebdy)[0][0] + deltaxy
            jjs = np.where(y12deg[:,0]>nbdy)[0][0] - deltaxy
            jjn = np.where(y12deg[:,0]>nbdy)[0][0] + deltaxy
        elif bbdy[ibdy] == 'east':
      #- 1/50 -
            msk50 = hFac50[:, :, -1]
            xx50 = tmpx50[:, -1]
            yy50 = tmpy50[:, -1]
            nxy50 = ny50
      #-  associated subdomain on the 1/12 grid -
            iiw = np.where(x12deg[0,:]>ebdy)[0][0] - deltaxy
            iie = np.where(x12deg[0,:]>ebdy)[0][0] + deltaxy
            jjs = np.where(y12deg[:,0]>sbdy)[0][0] - deltaxy
            jjn = np.where(y12deg[:,0]>nbdy)[0][0] + deltaxy
        elif bbdy[ibdy] == 'west':
      #- 1/50 -
            if varN[ivar] == 'uE':
                msk50 = hFac50[:, :, 1]         #obcs are applied at inner grid points
                xx50 = tmpx50[:, 1]
                yy50 = tmpy50[:, 1]
            else:
                msk50 = hFac50[:, :, 0]
                xx50 = tmpx50[:, 0]
                yy50 = tmpy50[:, 0]
            nxy50 = ny50
      #-  associated subdomain on the 1/12 grid -
            iiw = np.where(x12deg[0,:]>wbdy)[0][0] - deltaxy
            iie = np.where(x12deg[0,:]>wbdy)[0][0] + deltaxy
            jjs = np.where(y12deg[:,0]>sbdy)[0][0] - deltaxy
            jjn = np.where(y12deg[:,0]>nbdy)[0][0] + deltaxy
    #
    #- adjuste 1/50 land mask -
        msk50[np.where(msk50 > 0.0)] = 1.0
    #- select 1/12 subdomain coordinates (in [m]) for interpolation -
        [ny2, nx2] = [jjn-jjs, iie-iiw]
        xy12 = np.zeros([(ny2)*(nx2), 2])
        xy12[:, 0] = tmpx12[jjs:jjn,iiw:iie].reshape([ny2*nx2])
        xy12[:, 1] = tmpy12[jjs:jjn,iiw:iie].reshape([ny2*nx2])
    #
    #-- horizontal interpolation --
        print('Loading 1/12 field ...')
        var12 = np.zeros([nt+2, nr12, ny2, nx2])
        skip5D = 2
        da = open_mdsdataset(op.join(ddir,'RUNS/ORAR/memb%02d/run%4d/ocn/' 
                                     % (0,iper)), 
                             grid_dir=gdir,
                             iters=list(iters3),
                             prefix=['diag_ocnTave'], delta_t=2e2,
                            ).isel(YC=slice(jjs, jjn), YG=slice(jjs, jjn),
                                   XC=slice(iiw, iie), XG=slice(iiw, iie)
                                  )[varName[varOrder[ivar]]].drop_vars('time')
        da = xr.concat([da, open_mdsdataset(op.join(ddir,'RUNS/ORAR/memb%02d/run%4d/ocn/' 
                                                    % (0,iper-1)), 
                                            grid_dir=gdir,
                                            iters=list(iters2[skip5D:]),
                                            prefix=['diag_ocnTave'], delta_t=2e2,
                                           ).isel(YC=slice(jjs, jjn), YG=slice(jjs, jjn),
                                                  XC=slice(iiw, iie), XG=slice(iiw, iie)
                                                 )[varName[varOrder[ivar]]].drop_vars('time')],
                       dim='time')
        da.coords['time'] = np.concatenate((np.arange(73//2),np.arange(73//2+skip5D,73)))
        print(da)
        da = da.interp(time=np.arange(73))
        var12[:-2, :, :, :] = da.data
        var12[-1, :, :, :] = var12[0, :, :, :]
        var12[-2, :, :, :] = var12[-3, :, :, :]
    #- make some adjustments near land points (FOR TRACER ONLY) -
        if varN[ivar] == 't' or varN[ivar] == 's':
            for iit in range(nt+2):
                for kk in range(nr12):
                    for jj in range(ny2):
                        for ii in range(nx2-1, -1, -1):
                            if var12[iit, kk, jj, ii] == 0.0:
                                var12[iit, kk, jj, ii] = vlast
                            else:
                                vlast = var12[iit, kk, jj, ii]
    # #- Interpolate (with parallelization) -
    #     if __name__ == '__main__':
    #         p = Pool(nproc)
    #         tmp_interp = p.map(interp_obcs, np.arange(nt+2))
    # reshape
        var50 = np.zeros([nt+2, nr50, nxy50])
        for iit in range(nt+2):
            # var50[iit, :, :] = tmp_interp[iit].reshape([nr50, nxy50])
            var50[iit, :, :] = interp_obcs(iit).reshape([nr50, nxy50])
    #-- save --
        print('Save ...')
        f = open( str("%s/%s_%s_%04i.bin" % 
                      (tmpdir, varN[ivar], bbdy[ibdy]  ,iper) ), 'wb')
        var50.reshape([(nt+2)*nr50*nxy50]).astype('>f4').tofile(f)
        f.close()
    #
        del var12, var50
        # gc.collect()

== Deal with t at bdy south ==
Loading 1/12 field ...
<xarray.DataArray 'THETA' (time: 71, Z: 46, YC: 6, XC: 483)>
dask.array<concatenate, shape=(71, 46, 6, 483), dtype=float32, chunksize=(1, 46, 6, 483), chunktype=numpy.ndarray>
Coordinates: (12/13)
  * XC       (XC) >f4 278.2 278.3 278.4 278.4 278.5 ... 319.4 319.5 319.6 319.7
  * YC       (YC) >f4 25.33 25.42 25.5 25.58 25.67 25.75
  * Z        (Z) >f4 -3.047 -9.454 -16.36 ... -5.375e+03 -5.625e+03 -5.875e+03
    rA       (YC, XC) >f4 dask.array<chunksize=(6, 483), meta=np.ndarray>
    Depth    (YC, XC) >f4 dask.array<chunksize=(6, 483), meta=np.ndarray>
    drF      (Z) >f4 dask.array<chunksize=(46,), meta=np.ndarray>
    ...       ...
    hFacC    (Z, YC, XC) >f4 dask.array<chunksize=(46, 6, 483), meta=np.ndarray>
    maskC    (Z, YC, XC) bool dask.array<chunksize=(46, 6, 483), meta=np.ndarray>
    maskInC  (YC, XC) bool dask.array<chunksize=(6, 483), meta=np.ndarray>
    rhoRef   (Z) >f4 dask.array<chunksize=(46,), meta=np.ndarra